In [1]:
import os
import shutil
import random

# Directories
frames_dir = "C:/Users/LAMBDA THETA/Downloads/train/train/ann_images"  # Path to the folder containing frames
annotations_dir = "C:/Users/LAMBDA THETA/Downloads/train/train/labels"  # Path to the folder containing annotation files
output_dir = "C:/Users/LAMBDA THETA/Downloads/train/train/final"  # Path to save train/val splits

# Ensure reproducibility
random.seed(42)

# Create train and validation directories
train_frames_dir = os.path.join(output_dir, "train/images")
val_frames_dir = os.path.join(output_dir, "val/images")
train_annotations_dir = os.path.join(output_dir, "train/labels")
val_annotations_dir = os.path.join(output_dir, "val/labels")

os.makedirs(train_frames_dir, exist_ok=True)
os.makedirs(val_frames_dir, exist_ok=True)
os.makedirs(train_annotations_dir, exist_ok=True)
os.makedirs(val_annotations_dir, exist_ok=True)

# Get all frame and annotation filenames
frame_files = sorted([f for f in os.listdir(frames_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
annotation_files = sorted([f for f in os.listdir(annotations_dir) if f.lower().endswith('.txt')])

# Ensure matching files
assert len(frame_files) == len(annotation_files), "Mismatch between frames and annotation files."
assert all(os.path.splitext(f)[0] == os.path.splitext(a)[0] for f, a in zip(frame_files, annotation_files)), \
    "Frame and annotation file names do not match."

# Shuffle and split into train and validation sets
data = list(zip(frame_files, annotation_files))
random.shuffle(data)
split_idx = int(0.8 * len(data))
train_data = data[:split_idx]
val_data = data[split_idx:]

# Copy files to train and validation directories
for frame_file, annotation_file in train_data:
    shutil.copy(os.path.join(frames_dir, frame_file), os.path.join(train_frames_dir, frame_file))
    shutil.copy(os.path.join(annotations_dir, annotation_file), os.path.join(train_annotations_dir, annotation_file))

for frame_file, annotation_file in val_data:
    shutil.copy(os.path.join(frames_dir, frame_file), os.path.join(val_frames_dir, frame_file))
    shutil.copy(os.path.join(annotations_dir, annotation_file), os.path.join(val_annotations_dir, annotation_file))

print(f"Train and validation split completed.")
print(f"Train images saved in: {train_frames_dir}")
print(f"Validation images saved in: {val_frames_dir}")
print(f"Train annotations saved in: {train_annotations_dir}")
print(f"Validation annotations saved in: {val_annotations_dir}")


Train and validation split completed.
Train images saved in: C:/Users/LAMBDA THETA/Downloads/train/train/final\train/images
Validation images saved in: C:/Users/LAMBDA THETA/Downloads/train/train/final\val/images
Train annotations saved in: C:/Users/LAMBDA THETA/Downloads/train/train/final\train/labels
Validation annotations saved in: C:/Users/LAMBDA THETA/Downloads/train/train/final\val/labels


In [12]:
import os 
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2
from ultralytics import YOLO

detector = YOLO("runs/detect/train3/weights/best.pt")

video = "C:/Users/LAMBDA THETA/Downloads/test_bench_02/test_bench_02/Cam_19_03.mp4"#Cam_104_batch_5_46.mp4"
cap = cv2.VideoCapture(video)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    keypoint = YOLO("runs/pose/trail4/weights/best_y11.pt")
    frame = cv2.resize(frame, (1280,720))
    keypoint_results = keypoint.predict(frame)
    keypoints_results = keypoint_results[0]

    # for result in results:
            # keypoints = keypoints_results.keypoints
    if keypoints_results.keypoints is not None:
        keypoints_data = keypoints_results.keypoints.data
        for person_keypoints in keypoints_data:
            for keypoint in person_keypoints:
                x, y, confidence = keypoint[0].item(), keypoint[1].item(), keypoint[2].item()
                cv2.circle(frame, (int(x), int(y)), 7, (0, 255, 0), -1)

    pred = detector.predict(frame, )
    # pred_results = pred[0]

    # annotated_frame = pred_results.plot()
    annotated_frame = pred[0].plot()

    print("predictions:", pred)
    cv2.imshow("prediction: ", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
         break

cap.release()
cv2.destroyAllWindows()



    



0: 384x640 2 persons, 15.0ms
Speed: 1.0ms preprocess, 15.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 96x128 (no detections), 10.0ms
Speed: 0.0ms preprocess, 10.0ms inference, 0.0ms postprocess per image at shape (1, 3, 96, 128)
predictions: [ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'Hand_Raised'}
obb: None
orig_img: array([[[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  5,   0,   1],
        [  3,   0,   1],
        [  2,   0,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [224, 219, 220],
        [232, 229, 230],
        [236, 233, 234]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [238, 233, 234],
        [250, 247, 248],
        [254, 251, 252]],

       ...,

       [[128, 121, 106],
        [128, 

In [6]:
import os 
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

from ultralytics import YOLO

# Load a model
model = YOLO('yolov10s.pt')  # load a pretrained model (recommended for training)
# "C:\Users\LAMBDA THETA\Downloads\train\train\final\data.yaml"
# Train the model
results = model.train(data="C:/Users/LAMBDA THETA/Downloads/train/train/final/data.yaml", epochs=300, patience=30, imgsz=128, lr0=0.0001)

New https://pypi.org/project/ultralytics/8.3.51 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.18  Python-3.10.15 torch-2.5.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov10s.pt, data=C:/Users/LAMBDA THETA/Downloads/train/train/final/data.yaml, epochs=300, time=None, patience=30, batch=16, imgsz=128, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None

train: Scanning C:\Users\LAMBDA THETA\Downloads\train\train\final\train\labels... 6728 images, 677 backgrounds, 0 corrupt: 100%|██████████| 6728/6728 [00:20<00:00, 335.79it/s]


train: New cache created: C:\Users\LAMBDA THETA\Downloads\train\train\final\train\labels.cache


val: Scanning C:\Users\LAMBDA THETA\Downloads\train\train\final\val\labels... 1683 images, 200 backgrounds, 0 corrupt: 100%|██████████| 1683/1683 [00:05<00:00, 293.52it/s]

val: New cache created: C:\Users\LAMBDA THETA\Downloads\train\train\final\val\labels.cache


...

    300/300     0.424G     0.2726     0.1562      1.531          8        128: 100%|██████████| 421/421 [00:37<00:00, 11.35it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 53/53 [00:04<00:00, 11.17it/s]

                   all       1683       1483      0.991      0.998      0.995      0.993



300 epochs completed in 4.119 hours.
Optimizer stripped from runs\detect\train3\weights\last.pt, 16.5MB
Optimizer stripped from runs\detect\train3\weights\best.pt, 16.5MB

Validating runs\detect\train3\weights\best.pt...
Ultralytics 8.3.18  Python-3.10.15 torch-2.5.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLOv10s summary (fused): 293 layers, 8,035,734 parameters, 0 gradients, 24.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 53/53 [00:04<00:00, 12.58it/s]


                   all       1683       1483      0.993      0.998      0.995      0.994
Speed: 0.0ms preprocess, 0.4ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs\detect\train3
